# Importing libraries

In [35]:
import pandas as pd
import numpy as np
import math
import os
import sys
import json
import networkx as nx
from z3 import *

# Current and parent directories

In [37]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Read zone information

In [39]:
zones = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Zone-Info')
num_zones = len(zones)

# Parameters

In [41]:
num_timeslots = 1440
num_zones = 5

# Attack schedule

In [43]:
def attack_scheduling(list_time_min, list_time_max, start_time, final_time):
    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))
    num_timeslots = interval = 10
    
    
    for init_time in range(start_time, final_time, interval):
            
        cost = [Int( 'cost_' + str(i)) for i in range(num_zones)]
        zones = [Int( 'zones_' + str(i)) for i in range(num_zones)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(interval)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(interval)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(num_zones):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(num_zones):
            if list_time_min[z][init_time] == []:
                o.add(Implies(schedule[0] != prev_schedule, schedule[0] != z))

        for t in range(1, len(schedule)):
            for z in range(num_zones):
                if list_time_min[z][init_time + t] == []:
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(num_zones):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(num_zones):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(num_zones):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        o.maximize(total_cost)

        o.check()
        
        if o.check() == unsat:
            print('unsat', init_time)
            print(prev_stay, prev_schedule)
            return final_schedule, ultimate_cost
        
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))


        for t in range(interval): 
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
        
        
    return final_schedule, ultimate_cost

# Save linearized attack schedule (STRENGTH) for all houses and occupants

In [12]:
strength_attack_costs_actual = dict()

for adm_algo in ['DBSCAN']:
    for house_name in ['A']:
        for occupant_id in ['1', '2']:
            print("Linearized: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo))                  

            filename = str(parent_directory) + '/data/deadlock-elimination/Linearized_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(filename, "r") as file:
                data = json.load(file)

            rewards = [0, 1, 2, 4, 3]
            list_time_min = data["List-Time-Min"]
            list_time_max = data["List-Time-Max"]

            attack_schedule = attack_scheduling(list_time_min, list_time_max, 0, num_timeslots)[0]

            all_states = []
            arrival_time = 0
            arrival_zone = attack_schedule[0]
            stay_duration = 1

            for i in range(1, 1440):
                if attack_schedule[i] != attack_schedule[i - 1]:
                    all_states.append(str(arrival_time) + '-' + str(arrival_zone) + '-' + str(stay_duration))
                    stay_duration = 1
                    arrival_zone = attack_schedule[i]
                    arrival_time = i
                else:
                    stay_duration += 1

            if stay_duration > 1:
                all_states.append(str(arrival_time) + '-' + str(int(arrival_zone)) + '-' + str(stay_duration))  


            memory = {"All-States" : all_states, "Attack-Schedule" : attack_schedule[0], "Attack-Cost" : attack_schedule[1]}

            output_filename = str(parent_directory) + '/data/attack-schedules/shatter/Linearized_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(output_filename, "w") as json_file:
                json.dump(memory, json_file)

Linearized: House: A, Occupant: 1, ADM Algo: DBSCAN
0 0
10 0
20 0
30 0
40 0
50 0
60 0
70 0
80 0
90 0
100 0
110 0
120 0
130 0
140 0
150 0
160 0
170 0
180 0
190 0
200 0
210 0
220 0
230 0
240 0
250 0
260 0
270 0
280 0
290 0
300 0
310 0
320 0
330 0
340 0
350 0
360 0
370 0
380 0
390 0
400 0
410 0
420 0
430 0
440 0
450 0
460 0
470 0
480 0
490 0
500 0
510 0
520 0
530 0
540 0
550 0
560 0
570 0
580 0
590 0
600 0
610 0
620 0
630 0
640 0
650 0
660 0
670 0
680 0
690 18
700 34
710 20
720 0
730 0
740 0
750 0
760 0
770 0
780 0
790 0
800 0
810 0
820 0
830 0
840 0
850 0
860 0
870 0
880 0
890 0
900 0
910 0
920 0
930 0
940 0
950 0
960 0
970 0
980 0
990 0
1000 0
1010 0
1020 0
1030 0
1040 0
1050 0
1060 0
1070 0
1080 0
1090 0
1100 0
1110 0
1120 0
1130 0
1140 0
1150 0
1160 0
1170 0
1180 0
1190 0
1200 0
1210 0
1220 0
1230 0
1240 0
1250 0
1260 0
1270 0
1280 0
1290 0
1300 0
1310 0
1320 0
1330 0
1340 0
1350 0
1360 0
1370 0
1380 0
1390 0
1400 0
1410 0
1420 0
1430 0
Linearized: House: A, Occupant: 2, ADM Algo: DBS

In [13]:
attack_schedule[0]

0.0

# Save actual attack schedule (STRENGTH) for all houses and occupants

In [71]:
strength_attack_costs_actual = dict()

for adm_algo in ['DBSCAN']:
    for house_name in ['A']:
        for occupant_id in ['1', '2']:
            print("Actual: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo))                  

            filename = str(parent_directory) + '/data/deadlock-elimination/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(filename, "r") as file:
                data = json.load(file)

            rewards = [0, 1, 2, 4, 3]
            list_time_min = data["List-Time-Min"]
            list_time_max = data["List-Time-Max"]

            attack_schedule = attack_scheduling(list_time_min, list_time_max, 0, num_timeslots)

            all_states = []
            arrival_time = 0
            arrival_zone = attack_schedule[0]
            stay_duration = 1

            for i in range(1, 1440):
                if attack_schedule[0][i] != attack_schedule[0][i - 1]:
                    all_states.append(str(arrival_time) + '-' + str(arrival_zone) + '-' + str(stay_duration))
                    stay_duration = 1
                    arrival_zone = attack_schedule[0][i]
                    arrival_time = i
                else:
                    stay_duration += 1

            if stay_duration > 1:
                all_states.append(str(arrival_time) + '-' + str(int(arrival_zone)) + '-' + str(stay_duration))  


            memory = {"All-States" : all_states, "Attack-Schedule" : attack_schedule[0].tolist(), "Attack-Cost" : attack_schedule[1]}

            output_filename = str(parent_directory) + '/data/attack-schedules/shatter/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

            with open(output_filename, "w") as json_file:
                json.dump(memory, json_file)

Actual: House: A, Occupant: 1, ADM Algo: DBSCAN
0 0
10 0
20 0
30 0
40 0
50 0
60 0
70 0
80 0
90 0
100 0
110 0
120 0
130 0
140 0
150 0
160 0
170 0
180 0
190 0
200 0
210 0
220 0
230 0
240 0
250 0
260 0
270 0
280 0
290 0
300 0
310 0
320 0
330 0
340 0
350 0
360 0
370 0
380 0
390 0
400 0
410 0
420 0
430 0
440 0
450 0
460 0
470 0
480 0
490 0
500 16
510 30
520 40
530 24
540 20
550 20
560 20
570 20
580 20
590 20
600 20
610 20
620 20
630 20
640 20
650 20
660 20
670 20
680 20
690 20
700 20
710 20
720 20
730 20
740 20
750 20
760 20
770 20
780 20
790 20
800 20
810 20
820 20
830 20
840 20
850 20
860 20
870 20
880 20
890 20
900 20
910 20
920 20
930 20
940 20
950 20
960 20
970 20
980 20
990 20
1000 20
1010 20
1020 20
1030 20
1040 22
1050 30
1060 30
1070 36
1080 0
1090 0
1100 0
1110 0
1120 0
1130 0
1140 0
1150 0
1160 0
1170 0
1180 0
1190 0
1200 0
1210 0
1220 0
1230 0
1240 0
1250 0
1260 0
1270 0
1280 0
1290 0
1300 0
1310 0
1320 0
1330 0
1340 0
1350 0
1360 0
1370 0
1380 0
1390 0
1400 0
1410 0
1420 0
1430